## Chat with Tableau

Tableau's [VizQL Data Service](https://help.tableau.com/current/api/vizql-data-service/en-us/index.html) (aka VDS) provides developers with programmatic access to their Tableau Published Data Sources, allowing them to extend their business semantics for any custom workload or application, including AI Agents. The simple_datasource_qa tool adds VDS to the Langchain framework. This notebook shows you how you can use it to build agents that answer analytical questions grounded on your enterprise semantic models. 

Follow the [tableau-langchain](https://github.com/Tab-SE/tableau_langchain) project for more tools coming soon!


## Requirements
1. python version 3.12.2 or higher
2. A Tableau Cloud or Server environment with at least 1 published data source

Get started by installing and/or importing the required packages

In [ ]:
%pip install langchain-openai

In [ ]:
%pip install langgraph

In [ ]:
%pip install langchain-tableau --upgrade

Note you may need to restart your kernal to use updated packages

## Overview
The initialize_simple_datasource_qa initializes the Langgraph tool called [simple_datasource_qa](https://github.com/Tab-SE/tableau_langchain/blob/3ff9047414479cd55d797c18a78f834d57860761/pip_package/langchain_tableau/tools/simple_datasource_qa.py#L101), which can be used for analytical questions and answers on a Tableau Data Source.

This initializer function:
1. Authenticates to Tableau using Tableau's connected-app framework for JWT-based authentication. All the required variables must defined at runtime or as environment variables.
2. Asynchronously queries for the field metadata of the target datasource specified in the datasource_luid variable
3. Grounds on the metadata of the target datasource to transform natural language questions into the json-formatted query payload required to make VDS /query-datasource requests 
4. Executes a POST request to VDS
5. Formats and returns the results in a structured response

In [1]:
#langchain and langgraph package imports
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langgraph.prebuilt import create_react_agent

#langchain_tableau imports
#from langchain_tableau.tools.simple_datasource_qa import initialize_simple_datasource_qa
from experimental.tools.simple_datasource_qa import initialize_simple_datasource_qa

## Authentication Variables
You can declare your environment variables explicitly, as shown in several cases in this cookbook. However, ff these parameters are not provided, the simple_datasource_qa tool will attempt to automatically read them from environment variables.

For the Data Source that you choose, make sure you've updated the VizqlDataApiAccess permission in Tableau to allow the VDS API to access that Data Source via REST. More info [here](https://help.tableau.com/current/server/en-us/permissions_capabilities.htm#data-sources
). 

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

tableau_server = os.getenv('TABLEAU_DOMAIN') #replace with your Tableau server name
tableau_site = os.getenv('TABLEAU_SITE') #replace with your Tableau site
tableau_jwt_client_id = os.getenv('JWT_CLIENT_ID') #a JWT client ID (obtained through Tableau's admin UI)
tableau_jwt_secret_id = os.getenv('JWT_SECRET_ID') #a JWT secret ID (obtained through Tableau's admin UI)
tableau_jwt_secret = os.getenv('JWT_SECRET') #a JWT secret ID (obtained through Tableau's admin UI)
tableau_api_version = '3.21' #the current Tableau REST API Version
tableau_user = os.getenv('TABLEAU_USER') #replace with the username querying the target Tableau Data Source

# For this cookbook we are connecting to the Superstore dataset that comes by default with every Tableau server
datasource_luid = os.getenv('DATASOURCE_LUID') #the target data source for this Tool

# Add variables to control LLM models for the Agent and Tools
os.getenv("OPENAI_API_KEY") #set an your model API key as an environment variable
tooling_llm_model = os.getenv("TOOLING_MODEL") #set the LLM model for the Agent
print(f"Tooling LLM Model: {tooling_llm_model}")

Tooling LLM Model: gpt-4o


## Langgraph Example
This example uses the updated langgraph agent constructor class to achieve the same outcome. 

In [3]:
from IPython.display import display, Markdown

# Initalize simple_datasource_qa for querying Tableau Datasources through VDS
analyze_datasource = initialize_simple_datasource_qa(
    domain=tableau_server,
    site=tableau_site,
    jwt_client_id=tableau_jwt_client_id,
    jwt_secret_id=tableau_jwt_secret_id,
    jwt_secret=tableau_jwt_secret,
    tableau_api_version=tableau_api_version,
    tableau_user=tableau_user,
    datasource_luid=datasource_luid,
    tooling_llm_model=tooling_llm_model)

tools = [analyze_datasource]

model = ChatOpenAI(model='gpt-4o', temperature=0)

tableauAgent = create_react_agent(model, tools)

# Run the agent
messages = tableauAgent.invoke({"messages": [("human",'Rank schools by their average score in Writing where the score is greater than 499, showing their charter numbers.')]})
messages
#display(Markdown(messages['messages'][4].content)) #display a nicely formatted answer for successful generations

{'messages': [HumanMessage(content='Rank schools by their average score in Writing where the score is greater than 499, showing their charter numbers.', additional_kwargs={}, response_metadata={}, id='4d636238-0d6e-4cbd-a8a4-a436a8f0abb0'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_6axFIIfN4BBov8NbBpwEvIAo', 'function': {'arguments': '{"user_input":"Rank schools by their average score in Writing where the score is greater than 499, showing their charter numbers."}', 'name': 'simple_datasource_qa'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 486, 'total_tokens': 526, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_f5bdcc3276', 'id': 'chatcmpl-BSUHZiQIcO9LL25nm8F02Ezco8xO

In [4]:
display(Markdown(messages['messages'][2].content)) #display a nicely formatted answer for successful generations

text='\nThis is the output of a data query tool used to fetch information via Tableau\'s VizQL API\nYour task is to synthesize all of this information to provide a clear, concise answer to the end user.\n\nData Source Name: california_schools\nDescription: \nMaintainer: Mel Donaldson\n\nThis is the query written to Tableau\'s VizQL API:\n```json\n{\n  "fields": [\n    {\n      "fieldCaption": "School Name"\n    },\n    {\n      "fieldCaption": "Avgscrwrite",\n      "function": "AVG",\n      "sortDirection": "DESC",\n      "sortPriority": 1\n    },\n    {\n      "fieldCaption": "Charter School Number"\n    }\n  ],\n  "filters": [\n    {\n      "field": {\n        "fieldCaption": "Avgscrwrite"\n      },\n      "filterType": "QUANTITATIVE_NUMERICAL",\n      "quantitativeFilterType": "MIN",\n      "min": 500\n    }\n  ]\n}\n```\n\nThis is the resulting data from the query:\n| School Name | AVG(Avgscrwrite) | Charter School Number |\n| --- | --- | --- |\n| Mission San Jose High | 671.0 | None |\n| Lynbrook High | 664.0 | None |\n| Whitney (Gretchen) High | 661.0 | None |\n| Monta Vista High | 657.0 | None |\n| Saratoga High | 650.0 | None |\n| Henry M. Gunn High | 646.0 | None |\n| Oxford Academy | 634.0 | None |\n| Miramonte High | 633.0 | None |\n| Pacific Collegiate Charter | 630.0 | 0210 |\n| Piedmont High | 625.0 | None |\n| Palo Alto High | 624.0 | None |\n| University High | 624.0 | None |\n| Los Altos High | 622.0 | None |\n| Northwood High | 621.0 | None |\n| Cupertino High | 619.0 | None |\n| Troy High | 617.0 | None |\n| Torrey Pines High | 617.0 | None |\n| Campolindo High | 616.0 | None |\n| Dougherty Valley High | 613.0 | None |\n| San Marino High | 613.0 | None |\n| Canyon Crest Academy | 611.0 | None |\n| Palos Verdes Peninsula High | 607.0 | None |\n| Irvington High | 606.0 | None |\n| La Canada High | 605.0 | None |\n| Leland High | 603.0 | None |\n| Rancho Learning Center (Alternative) | 601.0 | None |\n| Academy of the Canyons | 596.0 | None |\n| Lowell High | 594.0 | None |\n| University High | 593.0 | 0890 |\n| Amador Valley High | 592.0 | None |\n| Arcadia High | 592.0 | None |\n| American High | 592.0 | None |\n| Homestead High | 591.0 | None |\n| Davis Senior High | 590.0 | None |\n| Mountain View High | 589.0 | None |\n| Los Gatos High | 589.0 | None |\n| Beverly Hills High | 588.0 | None |\n| Acalanes High | 587.0 | None |\n| Oak Park High | 586.0 | None |\n| Mira Loma High | 583.0 | None |\n| Irvine High | 583.0 | None |\n| OCSA | 582.0 | 0290 |\n| Westview High | 581.0 | None |\n| Alameda Community Learning Center | 581.0 | 0352 |\n| Redwood High | 579.0 | None |\n| Northcoast Preparatory and Performing Arts Academy | 579.0 | 0930 |\n| California Academy of Mathematics and Science | 579.0 | None |\n| Mira Costa High | 577.0 | None |\n| Woodbridge High | 577.0 | None |\n| Laguna Beach High | 577.0 | None |\n| William & Marian Ghidotti High | 576.0 | None |\n| Tamalpais High | 576.0 | None |\n| John Adams Academy | 576.0 | 1169 |\n| Menlo-Atherton High | 575.0 | None |\n| Carmel High | 575.0 | None |\n| Berkeley High | 574.0 | None |\n| Albany High | 574.0 | None |\n| Agoura High | 573.0 | None |\n| Westlake High | 573.0 | None |\n| Arnold O. Beckman High | 572.0 | None |\n| Aragon High | 572.0 | None |\n| California High | 571.0 | None |\n| L.A. County High School for the Arts | 570.0 | None |\n| Carlmont High | 569.0 | None |\n| Diamond Bar High | 568.0 | None |\n| San Ramon Valley High | 567.0 | None |\n| Technology High | 567.0 | None |\n| Monte Vista High | 566.0 | None |\n| Corona del Mar High | 566.0 | None |\n| Sir Francis Drake High | 566.0 | None |\n| Crescenta Valley High | 566.0 | None |\n| Northgate High | 566.0 | None |\n| Del Norte High | 565.0 | None |\n| Evergreen Valley High | 565.0 | None |\n| Scripps Ranch High | 564.0 | None |\n| Palos Verdes High | 564.0 | None |\n| South Pasadena Senior High | 563.0 | None |\n| Maria Carrillo High | 563.0 | None |\n| Inspire School of Arts and Sciences | 562.0 | 1114 |\n| Las Lomas High | 560.0 | None |\n| Burlingame High | 560.0 | None |\n| Tamiscal High (Alternative) | 559.0 | None |\n| San Mateo High | 559.0 | None |\n| Rancho Bernardo High | 559.0 | None |\n| Los Angeles Center for Enriched Studies | 559.0 | None |\n| Walnut High | 559.0 | None |\n| Da Vinci Charter Academy | 558.0 | 1079 |\n| Alta Vista Alternative High | 558.0 | None |\n| Temple City High | 556.0 | None |\n| Santa Susana High | 556.0 | None |\n| River Valley Charter | 555.0 | 0120 |\n| Alameda Science and Technology Institute | 555.0 | None |\n| Ocean Grove Charter | 554.0 | 0747 |\n| La Jolla High | 554.0 | None |\n| Calabasas High | 554.0 | None |\n| Malibu High | 553.0 | None |\n| San Dieguito High Academy | 553.0 | None |\n| Asawa (Ruth) San Francisco School of the Arts, A Public School. | 553.0 | None |\n| Poway High | 552.0 | None |\n| Dr. T. J. Owens Gilroy Early College Academy | 552.0 | None |\n| Vista del Lago High | 552.0 | None |\n| University Preparatory | 551.0 | 0612 |\n| Granite Bay High | 551.0 | None |\n| Sunny Hills High | 551.0 | None |\n| Petaluma High | 550.0 | None |\n| Santa Rosa High | 549.0 | None |\n| Foothill High | 549.0 | None |\n| Washington High | 549.0 | None |\n| San Luis Obispo High | 549.0 | None |\n| Classical Academy High | 548.0 | 0759 |\n| South High | 548.0 | None |\n| Aliso Niguel High | 548.0 | None |\n| Santa Cruz High | 547.0 | None |\n| Scotts Valley High | 547.0 | None |\n| Western Sierra Collegiate Academy | 547.0 | 1071 |\n| Cerritos High | 546.0 | None |\n| Shasta High | 546.0 | None |\n| Tesoro High | 545.0 | None |\n| Leigh High | 544.0 | None |\n| Analy High | 544.0 | None |\n| Granada Hills Charter High | 543.0 | 0572 |\n| Mills High | 543.0 | None |\n| Fountain Valley High | 543.0 | None |\n| Yorba Linda High | 543.0 | None |\n| Alameda High | 543.0 | None |\n| District Office | 542.4 | None |\n| Pacific Grove High | 542.0 | None |\n| Oak Ridge High | 542.0 | None |\n| Newbury Park High | 542.0 | None |\n| La Costa Canyon High | 542.0 | None |\n| Whitmore Charter High | 542.0 | 0658 |\n| Bret Harte Union High | 541.0 | None |\n| John A. Rowland High | 541.0 | None |\n| Branham High | 540.0 | None |\n| Arcata High | 540.0 | None |\n| Redwood Academy of Ukiah | 540.0 | 0271 |\n| Thousand Oaks High | 540.0 | None |\n| Bear River High | 540.0 | None |\n| Anderson W. Clark Magnet High | 539.0 | None |\n| West High | 539.0 | None |\n| Temecula Preparatory | 539.0 | 0284 |\n| Canyon High | 539.0 | None |\n| West Ranch High | 539.0 | None |\n| Piedmont Hills High | 538.0 | None |\n| Carlsbad High | 538.0 | None |\n| Everest Public High | 538.0 | 1070 |\n| Benicia High | 538.0 | None |\n| Folsom High | 538.0 | None |\n| Capistrano Valley High | 537.0 | None |\n| University Preparatory Academy Charter | 537.0 | 0844 |\n| El Camino High | 537.0 | None |\n| Middle College High | 537.0 | None |\n| Livermore High | 536.0 | None |\n| Mt. Carmel High | 536.0 | None |\n| Rio Americano High | 536.0 | None |\n| None | 535.9047619047619 | None |\n| Grove | 535.0 | 0180 |\n| Montgomery High | 535.0 | None |\n| California Connections Academy @ Ripon | 535.0 | 1398 |\n| Nevada Union High | 534.0 | None |\n| Edison High | 534.0 | None |\n| San Juan Hills High | 534.0 | None |\n| Huntington Beach High | 534.0 | None |\n| Visions In Education | 533.0 | 0248 |\n| Moorpark High | 533.0 | None |\n| Novato High | 533.0 | None |\n| Elise P. Buckingham Charter Magnet High | 533.0 | 0056 |\n| Dublin High | 533.0 | None |\n| Soquel High | 532.0 | None |\n| Coronado High | 532.0 | None |\n| Oceana High | 532.0 | None |\n| Prospect High | 531.0 | None |\n| Colfax High | 531.0 | None |\n| Guajome Park Academy Charter | 531.0 | 0050 |\n| Pioneer High | 531.0 | None |\n| Glen A. Wilson High | 531.0 | None |\n| Bella Vista High | 530.0 | None |\n| Foothill Technology High | 530.0 | None |\n| Placer High | 530.0 | None |\n| Granada High | 530.0 | None |\n| Valencia High | 530.0 | None |\n| Westmont High | 529.0 | None |\n| John F. Kennedy High | 529.0 | None |\n| Santa Barbara Senior High | 529.0 | None |\n| Dos Pueblos Senior High | 529.0 | None |\n| Terra Linda High | 529.0 | None |\n| Dehesa Charter | 529.0 | 0419 |\n| OCCS:CHEP/PCHS | 528.0 | None |\n| Mission Viejo High | 528.0 | None |\n| Dana Hills High | 527.0 | None |\n| Mendocino High | 527.0 | None |\n| Laguna Hills High | 527.0 | None |\n| Rocklin High | 527.0 | None |\n| Esperanza High | 527.0 | None |\n| Ponderosa High | 527.0 | None |\n| College Park High | 527.0 | None |\n| Charter Community School Home Study Academy | 527.0 | 0005 |\n| Mark Keppel High | 526.0 | None |\n| Adrian Wilcox High | 526.0 | None |\n| Sequoia High | 526.0 | None |\n| George Washington Carver School of Arts and Science | 526.0 | 0588 |\n| San Marin High | 526.0 | None |\n| Grossmont Middle College High | 525.0 | None |\n| Sherman Oaks Center for Enriched Studies | 525.0 | None |\n| Palisades Charter High | 525.0 | 0037 |\n| Glendora High | 525.0 | None |\n| Fremont High | 525.0 | None |\n| Casa Grande High | 525.0 | None |\n| Arroyo Grande High | 524.0 | None |\n| Natomas Charter | 524.0 | 0019 |\n| Half Moon Bay High | 524.0 | None |\n| Simi Valley High | 524.0 | None |\n| Castro Valley High | 524.0 | None |\n| Santa Monica High | 524.0 | None |\n| Oakland Charter High | 524.0 | 0883 |\n| Academy of the Redwoods | 524.0 | None |\n| William Howard Taft Charter High | 524.0 | 1580 |\n| Milpitas High | 524.0 | None |\n| San Diego International Studies | 523.0 | None |\n| Terra Nova High | 523.0 | None |\n| Alhambra Senior High | 523.0 | None |\n| Oakland School for the Arts | 523.0 | 0340 |\n| Venture (Alternative) | 523.0 | None |\n| Quincy Junior/Senior High | 523.0 | None |\n| San Clemente High | 522.0 | None |\n| Summit Preparatory Charter High | 522.0 | 0835 |\n| San Pasqual High | 522.0 | None |\n| California Virtual Academy @ Los Angeles | 521.0 | 0838 |\n| Chico High | 521.0 | None |\n| Atascadero High | 521.0 | None |\n| Templeton High | 521.0 | None |\n| El Toro High | 521.0 | None |\n| Adolfo Camarillo High | 521.0 | None |\n| Aptos High | 521.0 | None |\n| Mt. Everest Academy | 521.0 | None |\n| Brea-Olinda High | 521.0 | None |\n| Hillsdale High | 520.0 | None |\n| Stockdale High | 520.0 | None |\n| Marina High | 520.0 | None |\n| Preuss School UCSD | 520.0 | 0169 |\n| Redondo Union High | 520.0 | None |\n| Claremont High | 519.0 | None |\n| Stockton Unified Early College Academy | 519.0 | 1083 |\n| The High School at Moorpark College | 519.0 | None |\n| Clovis North High | 519.0 | None |\n| Del Oro High | 519.0 | None |\n| Saugus High | 518.0 | None |\n| Gabrielino High | 518.0 | None |\n| Trabuco Hills High | 517.0 | None |\n| California Virtual Academy @ San Diego | 517.0 | 0493 |\n| Julian Charter | 517.0 | 0267 |\n| Los Alamitos High | 517.0 | None |\n| San Lorenzo Valley High | 517.0 | None |\n| Windsor High | 516.0 | None |\n| Santa Ynez Valley Union High | 516.0 | None |\n| Valhalla High | 516.0 | None |\n| Sonora High | 515.5 | None |\n| Redlands Senior High | 515.0 | None |\n| Ukiah High | 515.0 | None |\n| Ann Sobrato High | 515.0 | None |\n| San Marcos Senior High | 514.0 | None |\n| Royal High | 514.0 | None |\n| Golden Sierra Junior Senior High | 514.0 | None |\n| El Camino Real Charter High | 514.0 | 1314 |\n| El Molino High | 514.0 | None |\n| Newport Harbor High | 514.0 | None |\n| American Indian Public High | 514.0 | 0765 |\n| Pleasant Grove High | 513.0 | None |\n| Mariposa County High | 513.0 | None |\n| Escondido Charter High | 513.0 | 0109 |\n| Burbank High | 513.0 | None |\n| El Modena High | 513.0 | None |\n| El Segundo High | 513.0 | None |\n| Saint Helena High | 513.0 | None |\n| Cypress High | 513.0 | None |\n| Mt. Shasta High | 513.0 | None |\n| Capistrano Connections Academy | 512.0 | 0664 |\n| C. K. McClatchy High | 512.0 | None |\n| Franklin High | 512.0 | None |\n| Morro Bay High | 512.0 | None |\n| Burroughs High | 511.5 | None |\n| Van Nuys Senior High | 511.0 | None |\n| High Tech High North County | 511.0 | 0756 |\n| Los Osos High | 511.0 | None |\n| Ferndale High | 511.0 | None |\n| Heritage High | 511.0 | None |\n| Silver Creek High | 511.0 | None |\n| Tahoe Truckee High | 511.0 | None |\n| Fall River Junior-Senior High | 511.0 | None |\n| Herbert Hoover High | 511.0 | None |\n| El Dorado High | 510.0 | None |\n| Oakmont High | 510.0 | None |\n| William S. Hart High | 510.0 | None |\n| North Hollywood Senior High | 510.0 | None |\n| Ruben S. Ayala High | 510.0 | None |\n| Grover Cleveland Charter High | 509.0 | 1571 |\n| Harbor Teacher Preparation Academy | 509.0 | None |\n| Villa Park High | 509.0 | None |\n| North Tahoe High | 508.0 | None |\n| High Tech LA | 508.0 | 0537 |\n| Ventura High | 508.0 | None |\n| Palm Desert High | 508.0 | None |\n| San Marcos High | 508.0 | None |\n| Healdsburg High | 508.0 | None |\n| Forest Charter | 508.0 | 1427 |\n| Pleasant Valley High | 508.0 | None |\n| Santa Teresa High | 508.0 | None |\n| Chino Hills High | 508.0 | None |\n| Enterprise High | 507.0 | None |\n| Buchanan High | 507.0 | None |\n| Clovis West High | 507.0 | None |\n| Renaissance Arts Academy | 506.0 | 0579 |\n| Murrieta Valley High | 506.0 | None |\n| Cypress Charter High | 506.0 | 0512 |\n| Point Loma High | 506.0 | None |\n| Academy for Academic Excellence | 505.0 | 0127 |\n| Great Oak High | 505.0 | None |\n| FAME Public Charter | 505.0 | 0728 |\n| Nordhoff High | 505.0 | None |\n| Cabrillo High | 504.0 | None |\n| Amador High | 504.0 | None |\n| Independence High | 504.0 | None |\n| Santa Clara High | 504.0 | None |\n| Connecting Waters Charter | 504.0 | 0477 |\n| Vanden High | 504.0 | None |\n| Paradise Senior High | 504.0 | None |\n| Oakland Technical High | 503.0 | None |\n| Aspire Benjamin Holt College Preparatory Academy | 503.0 | 0565 |\n| Santa Rosa Academy | 503.0 | 0730 |\n| San Rafael High | 503.0 | None |\n| Whitney High | 502.0 | None |\n| Argonaut High | 502.0 | None |\n| Eureka Senior High | 502.0 | None |\n| Oakdale High | 502.0 | None |\n| Culver City High | 502.0 | None |\n| Valley Oaks Charter | 502.0 | 0332 |\n| San Francisco Flex Academy | 502.0 | 1208 |\n| City of Angels | 501.0 | None |\n| Liberty High | 501.0 | None |\n| Ramona High | 501.0 | None |\n| Sierra High | 501.0 | None |\n| University City High | 501.0 | None |\n| Clayton Valley Charter High | 501.0 | 1400 |\n| Tehachapi High | 501.0 | None |\n| Lodi High | 501.0 | None |\n| Woodcreek High | 501.0 | None |\n| Mission Vista High | 501.0 | None |\n| Santana High | 501.0 | None |\n| University Preparatory | 500.0 | None |\n| Golden Valley High | 500.0 | None |\n| West Campus | 500.0 | None |\n| Napa High | 500.0 | None |\n| Bonita High | 500.0 | None |\n| Vasquez High | 500.0 | None |\n| New Technology High | 500.0 | None |\n| John C. Kimball High | 500.0 | None |\n| Torrance High | 500.0 | None |\n| Modoc High | 500.0 | None |\n| Yosemite High | 500.0 | None |\n\n\nThis was the user_input (question or task):\nRank schools by their average score in Writing where the score is greater than 499, showing their charter numbers.\n\nBased on the provided context, formulate a comprehensive and informative response to the user\'s query.\nYour response should:\n1. Describe the data source name, description and maintainer if this is the first interaction the user has with it\n2. Use the resulting data to answer the user\'s question or task\n3. Be short and concise, if the data table is too long only return the relevant rows or a small sample\n\nYour synthesized response:\n'